<img src="https://raw.githubusercontent.com/Torsion-Audio/Scyclone/main/assets/pictures/interface.png" width="100%"/>

__Scyclone__ is an audio plugin developed by **Torsion Audio** that utilizes neural timbre transfer technology to offer a new approach to audio production. The plugin builds upon [RAVE](https://github.com/acids-ircam/RAVE) [Antoine Cailon et. al], a realtime audio variational auto encoder, facilitating neural timbre transfer in both single and couple inference mode.

This enables a new artificial layering technique to be applied on the incoming signal in creating richer drum pallets, fuller atmospheres or simply transferring the timbre of the raw signal to another sound pallet. To further control the behaviour and production of the neural networks, we have internally equipped the plugin with signal processing modules allowing the user to shape, control and embellish the source and target timbres in a distinct manner.

Scyclone comes with two pre-trained models, **funk_drums** trained on four hours of data inspired by the captivating sounds of vintage drum-breaks and **Djembe**, trained on five hours of carefully compiled Djembe dataset.


This notebook allows you to train your own models on customized datasets to use as presets inside the plugin. Although we encorage the interested reader to visit RAVE [article](https://arxiv.org/pdf/2111.05011.pdf) and study the well-written [github](https://github.com/acids-ircam/RAVE) repository.


There are many ways to train machine learning models both on local machines and on cloud. To keep things simple for you, we chose Google Colab as it offeres an all-in-one solution to the problem. Please note that we do not endorse for Google or any other cloud computing service provider.

<img src="https://raw.githubusercontent.com/Torsion-Audio/Scyclone/main/assets/pictures/logo.png" alt="alt" width="1%"/>  **`Torsion Audio`**














In [ ]:
#@title Get System Specs
!nvidia-smi

In [ ]:
#@title Mount Google Drive
from google.colab import drive
drive.mount('content')

In [ ]:
#@title Install Dependencies

print('This might take several minutes...')
!curl -L https://repo.anaconda.com/miniconda/Miniconda3-py39_4.12.0-Linux-x86_64.sh -o miniconda.sh
!chmod +x miniconda.sh
!sh miniconda.sh -b -p /content/miniconda
!/content/miniconda/bin/pip install --quiet acids-rave==2.1.7
!/content/miniconda/bin/pip install --quiet pytorch_lightning==1.9.0
!/content/miniconda/bin/pip install cached-conv==2.4.0
!/content/miniconda/bin/pip install onnx
!/content/miniconda/bin/pip install --quiet --upgrade ipython ipykernel
!/content/miniconda/bin/conda install ffmpeg -y
!/content/miniconda/bin/apt-get install unzip
!nvidia-smi

In [ ]:
#@title Unzip data
import os

data_zip = ""     #@param {type:"string"}
dataset_dir = ""  #@param {type:"string"}

!unzip data_zip -d dataset_dir

In [ ]:
#@title Train

name = ""           #@param {type:"string"}
dataset_dir = ""    #@param {type:"string"}
save_dir = ""       #@param {type:"string"}
architecture = "v1" #@param ["v1", "onnx"]
sr = 48000       #@param [48000, 44100]
CAPACITY = 32     #@param {type:"number"}
LATENT_SIZE = 16 #@param {type:"number"}


%cd /content/
!mkdir dataset
%cd $save_dir
preprocessed_dataset = "/content/dataset"


!/content/miniconda/bin/rave preprocess --input_path $dataset --output_path $preprocessed_dataset --sampling_rate sr
!/content/miniconda/bin/rave train --config $architecture --config $regularization --config centered --db_path $preprocessed_dataset --name $name --override LATENT_SIZE=LATENT_SIZE --override CAPACITY=CAPACITY

In [ ]:
#@title Resume 

name = ""           #@param {type:"string"}
dataset_dir = ""    #@param {type:"string"}
save_dir = ""       #@param {type:"string"}
architecture = "v1" #@param ["v1", "onnx"]
sr = 48000       #@param [48000, 44100]
CAPACITY = 32     #@param {type:"number"}
LATENT_SIZE = 16 #@param {type:"number"}
resume_dir = "" #@param {type:"string"}


%cd /content/
!mkdir dataset
%cd $save_dir
preprocessed_dataset = "/content/dataset"


# !/content/miniconda/bin/rave train --helpfull
!/content/miniconda/bin/rave preprocess --input_path $dataset --output_path $preprocessed_dataset --sampling_rate sr
!/content/miniconda/bin/rave train --config $architecture --config $regularization --db_path $preprocessed_dataset --name $name --override LATENT_SIZE=16 --override CAPACITY=32

In [ ]:
#@title Export Model to ONNX

run_dir = "" #@param {type:"string"}
!/content/miniconda/bin/rave export_onnx --run $run_dir